In [1]:
import sunpy.map
from sunpy.net import Fido, attrs as a
import astropy.units as u
from datetime import datetime, timedelta

# # Step 1: Time range 
# now = datetime.utcnow()
# time_ago = now - timedelta(minutes=60*2)
# time_range = a.Time(time_ago.isoformat(), now.isoformat())

# # Step 2: Query AIA 171Å data
# results = Fido.search(time_range, a.Instrument.aia, a.Wavelength(171 * u.angstrom))
# downloaded_files = Fido.fetch(results)

# # Step 3: Load map sequence
# maps = sunpy.map.Map(downloaded_files, sequence=True)


In [4]:
# Step 1: Time range 
now = datetime.utcnow() - timedelta(days=26)
time_ago = now - timedelta(hours=1)
time_range = a.Time(time_ago.isoformat(), now.isoformat())

C:\Users\bassd\AppData\Local\Temp\ipykernel_29012\4028400222.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow() - timedelta(days=26)


In [3]:
time_range

<sunpy.net.attrs.Time(2025-04-16 17:32:59.084, 2025-04-16 22:20:59.084)>

In [1]:
# Step 2: Query AIA 171Å data
results = Fido.search(time_range, a.Instrument.aia, a.Wavelength(171 * u.angstrom))
downloaded_files = Fido.fetch(results)

# Step 3: Load map sequence
maps = sunpy.map.Map(downloaded_files, sequence=True)

In [ ]:
import imageio.v2 as imageio  # ensure compatibility with latest imageio
import numpy as np
from IPython.display import Image as IPyImage

# Step 4: Create grayscale images from SunPy maps
frames = []
for m in maps:
    data = m.data.astype(float)

    # Avoid log(0) and negatives
    data[data <= 0] = 1e-3

    # Apply log10 scaling
    log_data = np.log10(data)

    # Optional: manually adjust dynamic range here
    lower = 1  # black point (adjust if needed)
    upper = 4  # white point (adjust if needed)

    # Normalize manually
    normed = (log_data - lower) / (upper - lower)
    normed = np.clip(normed, 0, 1)

    # Convert to 8-bit grayscale
    image = (255 * normed).astype(np.uint8)
    frames.append(image)

# Step 5: Save as animated GIF
gif_path = "sun_sequence.gif"
imageio.mimsave(gif_path, frames, duration=0.01, loop=0)  # duration = seconds per frame

# Step 6: Display the GIF in Jupyter
IPyImage(filename=gif_path)


In [ ]:
data_i

In [ ]:
gif_path = "sun_sequence_f.gif"
imageio.mimsave(gif_path, frames, duration=0.0001, loop=0)  # duration = seconds per frame

# Step 6: Display the GIF in Jupyter
IPyImage(filename=gif_path)

In [ ]:
import os
from datetime import datetime, timedelta
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from sunpy.map import Map
from sunpy.net import Fido, attrs as a
from sunpy.visualization.colormaps import color_tables as ct

In [ ]:
output_dir = "sun-frames"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
end_time = datetime.utcnow() - timedelta(days=1)
start_time = end_time - timedelta(days=27)

In [ ]:
time_list = [start_time + timedelta(hours=3 * i) for i in range(int(27 * 24 / 3))]

In [ ]:
# Generate list of sample times: every 3 hours
time_list = [start_time + timedelta(hours=3 * i) for i in range(int(27 * 24 / 3))]

# Download and save images
for i, t in enumerate(time_list):
    try:
        # Query AIA 171Å image closest to this time
        tr = a.Time(t, t + timedelta(minutes=10))
        result = Fido.search(tr, a.Instrument.aia, a.Wavelength(171 * u.angstrom))
        file = Fido.fetch(result[0, 0])

        # Load map
        m = Map(file[0])

        # Apply SunPy's AIA 171 colormap and intensity normalization
        aia_cmap = ct.aia_color_table(171 * u.angstrom)
        norm = plt.Normalize(vmin=0, vmax=np.percentile(m.data, 99.5))  # Stretch using 99.5 percentile

        # Save colorized image
        fig = plt.figure(figsize=(6, 6), dpi=100)
        ax = plt.axes([0, 0, 1, 1])  # no border/margin
        ax.imshow(m.data, cmap=aia_cmap, norm=norm, origin='lower')
        ax.axis('off')
        plt.savefig(f"{output_dir}/frame_{i:03d}.png", bbox_inches='tight', pad_inches=0)
        plt.show()
        plt.close(fig)

        print(f"Saved frame_{i:03d}.png")
        break

    except Exception as e:
        print(f"Skipped time {t} due to error: {e}")
        break


In [ ]:
from datetime import datetime, timedelta
from sunpy.net import Fido, attrs as a
import astropy.units as u
import sunpy.map

In [ ]:
# Time range
now = datetime.utcnow()
start = now - timedelta(days=27)
time_range = a.Time(start.isoformat(), now.isoformat())

In [ ]:
now

In [ ]:
start

In [ ]:
# Fetch many images at once
results = Fido.search(time_range, a.Instrument.aia, a.Wavelength(171 * u.angstrom))
downloaded_files = Fido.fetch(results)

# Load all as a sequence
maps = sunpy.map.Map(downloaded_files, sequence=True)

# Downsample by taking every Nth frame
sampled_maps = maps[::10]  # Adjust the stride as needed

In [ ]:
for y in [maps[x].date for x in range(len(maps))]:
    print(y)

In [ ]:
import os
import matplotlib.pyplot as plt
from sunpy.visualization.colormaps import color_tables as ct
import astropy.units as u
import numpy as np
from astropy.visualization import LogStretch, ImageNormalize
from sunpy.visualization.colormaps import color_tables as ct

In [ ]:
output_dir = "sun-frames"

# Get the AIA 171 colormap
aia_cmap = ct.aia_color_table(171 * u.angstrom)

for i, m in enumerate(sampled_maps):
    try:
        data = m.data

        # Normalize with 99.5th percentile stretch
        #vmin = 0
        #vmax = np.percentile(data, 96)
        #norm = plt.Normalize(vmin=vmin, vmax=vmax)
        norm = ImageNormalize(data, stretch=LogStretch(), vmin=8, vmax=5000)

        # Create figure with no axes
        fig = plt.figure(figsize=(6, 6), dpi=100)
        ax = plt.axes([0, 0, 1, 1])  # full-figure, no margins
        #ax.imshow(data, cmap=aia_cmap, norm=norm, origin='lower')
        ax.imshow(data, cmap=aia_cmap, norm=norm, origin='lower')
        ax.axis('off')

        # Save to file
        frame_path = os.path.join(output_dir, f"frame_{i:03d}.png")
        plt.savefig(frame_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        print(f"Saved {frame_path}")

    except Exception as e:
        print(f"Skipping frame {i} due to error: {e}")


In [14]:
import sunpy.map
from sunpy.visualization.colormaps import color_tables as ct
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import os
from scipy.ndimage import zoom
from matplotlib.cm import get_cmap

In [2]:
input_dir = 'downloaded-fits'
output_dir = 'sun-frames'
#os.makedirs(output_dir, exist_ok=True)

files = sorted([f for f in os.listdir(input_dir) if f.endswith('lev1.fits')])

for i, fname in enumerate(files):
    m = sunpy.map.Map(os.path.join(input_dir, fname))
    data = m.data.astype(float)

    # Log scaling for better contrast
    data[data <= 0] = 1e-3
    log_data = np.log10(data)
    normed = np.clip((log_data - 1.5) / (4.2 - 1.5), 0, 1)

    scale = 512 / normed.shape[0]
    resized = zoom(normed, zoom=scale, order=3) 

    # Apply AIA colormap
    cmap = get_cmap(ct.aia_color_table(171 * u.angstrom))
    rgba = cmap(resized)

    plt.imsave(f"{output_dir}/frame_{i:03d}.png", rgba)
    print(f"Saved frame {i}")


In [9]:
from astropy.io import fits
hdul = fits.open('downloaded-fits//aia.lev1_euv_12s.2024-04-14T235934Z.171.image_lev1.fits')
hdul.info()

Filename: downloaded-fits//aia.lev1_euv_12s.2024-04-14T235934Z.171.image_lev1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  COMPRESSED_IMAGE    1 CompImageHDU    196   (4096, 4096)   int16   
